# 如何使用输出解析器将 LLM 响应解析为结构化格式

语言模型输出文本。但有时您希望获取比纯文本更结构化的信息。虽然一些模型提供商支持[返回结构化输出的内置方法](/docs/how_to/structured_output)，但并非所有提供商都支持。

[输出解析器](/docs/concepts/output_parsers/) 是帮助结构化语言模型响应的类。输出解析器必须实现两个主要方法：

- "Get format instructions"：一个返回字符串的方法，其中包含有关如何格式化语言模型输出的说明。
- "Parse"：一个接收字符串（假定为语言模型的响应）并将其解析为某种结构的方法。

然后还有一个可选方法：

- "Parse with prompt"：一个接收字符串（假定为语言模型的响应）和提示（假定为生成该响应的提示）并将其解析为某种结构的方法。提供提示主要是为了在 OutputParser 想要重试或修复输出时，需要提示中的信息来执行此操作。

## 入门

下面我们介绍主要的输出解析器类型 `PydanticOutputParser`。

In [1]:
from langchain_core.output_parsers import PydanticOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_openai import OpenAI
from pydantic import BaseModel, Field, model_validator

model = OpenAI(model_name="gpt-3.5-turbo-instruct", temperature=0.0)


# Define your desired data structure.
class Joke(BaseModel):
    setup: str = Field(description="question to set up a joke")
    punchline: str = Field(description="answer to resolve the joke")

    # You can add custom validation logic easily with Pydantic.
    @model_validator(mode="before")
    @classmethod
    def question_ends_with_question_mark(cls, values: dict) -> dict:
        setup = values.get("setup")
        if setup and setup[-1] != "?":
            raise ValueError("Badly formed question!")
        return values


# Set up a parser + inject instructions into the prompt template.
parser = PydanticOutputParser(pydantic_object=Joke)

prompt = PromptTemplate(
    template="Answer the user query.\n{format_instructions}\n{query}\n",
    input_variables=["query"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

# And a query intended to prompt a language model to populate the data structure.
prompt_and_model = prompt | model
output = prompt_and_model.invoke({"query": "Tell me a joke."})
parser.invoke(output)

Joke(setup='Why did the tomato turn red?', punchline='Because it saw the salad dressing!')

## LCEL

输出解析器实现了 [Runnable 接口](/docs/concepts/runnables)，这是 [LangChain Expression Language (LCEL)](/docs/concepts/lcel) 的基本构建块。这意味着它们支持 `invoke`、`ainvoke`、`stream`、`astream`、`batch`、`abatch`、`astream_log` 调用。

输出解析器接受字符串或 `BaseMessage` 作为输入，并可以返回任意类型。

In [2]:
parser.invoke(output)

Joke(setup='Why did the tomato turn red?', punchline='Because it saw the salad dressing!')

我们可以不手动调用解析器，而是将其添加到我们的 `Runnable` 序列中：

In [3]:
chain = prompt | model | parser
chain.invoke({"query": "Tell me a joke."})

Joke(setup='Why did the tomato turn red?', punchline='Because it saw the salad dressing!')

尽管所有解析器都支持流式接口，但只有特定的解析器能够通过部分解析的对象进行流式传输，因为这在很大程度上取决于输出类型。无法构造部分对象的解析器将直接产生完全解析的输出。

例如，`SimpleJsonOutputParser` 可以通过部分输出来进行流式传输：

In [4]:
from langchain.output_parsers.json import SimpleJsonOutputParser

json_prompt = PromptTemplate.from_template(
    "Return a JSON object with an `answer` key that answers the following question: {question}"
)
json_parser = SimpleJsonOutputParser()
json_chain = json_prompt | model | json_parser

In [5]:
list(json_chain.stream({"question": "Who invented the microscope?"}))

[{},
 {'answer': ''},
 {'answer': 'Ant'},
 {'answer': 'Anton'},
 {'answer': 'Antonie'},
 {'answer': 'Antonie van'},
 {'answer': 'Antonie van Lee'},
 {'answer': 'Antonie van Leeu'},
 {'answer': 'Antonie van Leeuwen'},
 {'answer': 'Antonie van Leeuwenho'},
 {'answer': 'Antonie van Leeuwenhoek'}]

类似地，对于 `PydanticOutputParser`：

In [6]:
list(chain.stream({"query": "Tell me a joke."}))

[Joke(setup='Why did the tomato turn red?', punchline=''),
 Joke(setup='Why did the tomato turn red?', punchline='Because'),
 Joke(setup='Why did the tomato turn red?', punchline='Because it'),
 Joke(setup='Why did the tomato turn red?', punchline='Because it saw'),
 Joke(setup='Why did the tomato turn red?', punchline='Because it saw the'),
 Joke(setup='Why did the tomato turn red?', punchline='Because it saw the salad'),
 Joke(setup='Why did the tomato turn red?', punchline='Because it saw the salad dressing'),
 Joke(setup='Why did the tomato turn red?', punchline='Because it saw the salad dressing!')]